In [29]:
# imports for the shake of importing

import keras
import tensorflow as tf
from tensorflow import nn 
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.applications import NASNetLarge
from keras import models, layers, optimizers, backend
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator

# THICC networks ftw?

My GPU is hating me, I really wish I had a RTX card right now >_>

In [25]:
img_height, img_width = 331, 331  # For NASNetLarge
classnames = ["Background", "BMP2", "Buk-M1-2", "T14", "T90", "ZSU23"]
classes = len(classnames)

## Level 1

#### Copied directly from Ugur's code

Made some modifications fo that it would fit better in a notebook

In [26]:
conv_base1 = NASNetLarge(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

model = models.Sequential()
model.add(conv_base1)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 4032)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                129056    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 198       
Total params: 85,046,072
Trainable params: 84,849,404
Non-trainable params: 196,668
_________________________________________________________________


In [27]:
# freezing the base
conv_base1.trainable = False

# compile
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.RMSprop(lr=0.0002, decay=0.0001), 
              metrics=['acc'])

# never used this before, should be interesting
callbacks_list_L1 = [ModelCheckpoint(filepath='./weights', 
                                     save_weights_only=True, 
                                     monitor='val_acc', 
                                     verbose=1, 
                                     save_best_only=True),
                                     ReduceLROnPlateau(monitor='val_acc', 
                                     factor=0.5, 
                                     patience=1, 
                                     verbose=1),
                                     TensorBoard(log_dir='./TensorBoard/L1')]

In [31]:
# data STUFF

batch = 64

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    './train',
    target_size=(img_height, img_width),
    batch_size=batch,
    shuffle=True,
    classes=classnames,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    './test',
    target_size=(img_height, img_width),
    batch_size=1,
    shuffle=True,
    classes=classnames,
    class_mode='categorical')

Found 42000 images belonging to 6 classes.
Found 90 images belonging to 6 classes.


In [35]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=(42000 * classes) // batch,
        epochs=1,
        callbacks=callbacks_list_L1,
        validation_data=test_generator,
        validation_steps=90,
        verbose=1)

Epoch 1/1
   1/3937 [..............................] - ETA: 35:41:24 - loss: 1.9142 - acc: 0.1562

KeyboardInterrupt: 

In [39]:
def plot(h,t,e):
    history_dict = h[0]
    loss_values = history_dict['loss']
    validation_loss_values = history_dict['val_loss']
    acc_values = history_dict['acc']
    validation_acc_values = history_dict['val_acc']
    epochs_range = range(1, e + 1)

    # Plotting Training and Validation loss of the corresponding Model
    plt.plot(epochs_range, loss_values, 'bo', label='Training loss')
    plt.plot(epochs_range, validation_loss_values, 'ro', label='Validation loss')
    plt.title('Training and validation loss of ' + t)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.yticks(np.arange(0, 3.1, step=0.2))
    plt.legend()
    plt.show()

    # Plotting Training and Validation accuracy of the corresponding Model
    plt.plot(epochs_range, acc_values, 'bo', label='Training accuracy')
    plt.plot(epochs_range, validation_acc_values, 'ro', label='Validation accuracy')
    plt.title('Training and validation accuracy of ' + t)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.yticks(np.arange(0.3, 1.1, step=0.1))
    plt.legend()
    plt.show()

In [ ]:
plot(history.history, "LEVEL1:", epochsL1)